In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from simple_radio_script_encapsulated import _get_teaser_for_story, _get_statements_for_story_id
from simple_radio_script_encapsulated import teaser_matches_crit, aggregate_decisions
from IPython.display import display, Markdown

In [3]:
#story_no = 22192 # 23183 # 23188 # 22192 # 23168
story_no = 23168
#story_no = 23183

In [4]:
t1 = _get_teaser_for_story(story_no)

In [5]:
statements = _get_statements_for_story_id(story_no)

In [6]:
crit_match_prompt = """You are a professional journalist in the medicine domain. You will receive a text and i want you to decide whether the following criterion matches the text.
To help you guide your decision, you will also get a few expert statements in JSON Form.

Please answer in the following format:
Criterion: One sentence summary of the criterion.
Decision: yes or no
References: Paragraphs in the text which match the criterion.

For the references find all text passages and quote them in the form of a list.
Indicate for all text passages whether they come from the text or from an expert statement. 
If it is a statement, you must explicitely name the expert who made the statement.

```
CRITERION:
{CRITERIA}
```

```
TEXT: 
{TEASER}
```

```
EXPERT STATEMENTS: 
{STATEMENTS}
```
You must answer in German.
""" 

In [30]:
crit1 = """Der Text erwähnt den Evidenzlevel. 
Was sind die Einschränkungen bei den Studienergebnissen? 
Gibt es Einschränkungen, die dazu führen, dass die zentralen Ergebnisse fragwürdig sind? 
"""
crit2 = "The text contains information about cancer"
crit3 = "The text contains mentions a specifoc journal name of a study"
crit4 = "The text describes a mathematical method for optimizing a loss function with four objectives"
crit5 = """Der Text macht klar, in welchem Forschungsstadium sich die Ergebnisse befinden. 
Dies lässt Rückschlüsse auf die Verfügbarkeit von beispielsweise neuen Medikamenten oder Behandlungen zu."""
crit6 = """Der Text erwähnt das zentrale Forschungsergebnis"""

crits = [crit1, crit2, crit3, crit4, crit5, crit6]

In [31]:
outputs = [teaser_matches_crit(t1, str(statements), crit_match_prompt, c) for c in crits]

In [32]:
outputs

['KRITERIUM:\nDer Text erwähnt den Evidenzlevel. Was sind die Einschränkungen bei den Studienergebnissen? Gibt es Einschränkungen, die dazu führen, dass die zentralen Ergebnisse fragwürdig sind?\n\nENTSCHEIDUNG:\nJa\n\nREFERENZEN:\n- "Dabei stellten sie fest, dass sich bei geringer Viruslast neun von zehn Hühner nicht infizierten, bei steigender Viruslast die Resistenz jedoch abnahm." (aus dem Text)\n- "Eine vollständige Resistenz der Hühner könnte theoretisch erreicht werden, wenn alle drei Gene der Genfamilie ANP32 verändert werden würden. Das zeigten Experimente in Zellkulturen. Das Ausschalten aller drei Gene wäre für ein Huhn jedoch vermutlich tödlich." (aus dem Text)\n- "Doch auch hier gibt es Einschränkungen: In dem Leitfaden für die Umweltverträglichkeitsprüfung von gentechnisch veränderten Tieren der EFSA [III] wird auf diverse Risiken hingewiesen: Genetisch veränderte Tiere können eine längere Inkubationszeit haben und sich auch mit Krankheitserregern aktiver verhalten, wodur

In [37]:
agg_decisions_prompt = """
Du wirst als Input eine Liste bekommen.
Jedes Element der Liste besteht aus einem Kriterium, einer binären Entscheidung und einigen Referenzen.

Formatiere den Input als Markdown Tabelle. Antworte ausschließlich mit der Tabelle.
Das Ergebnis soll etwa so aussehen:
| Kriterium | Entscheidung | Referenzen |
| ------------- |:--------:| -------------------------------------------------- :|
| Kriterium 1 | ja | <ul><li>Referenz 1</li><li>Referenz 2</li></ul> |
| Kriterium 2 | nein | <ul><li>Referenz 1</li><li>Referenz 2</li></ul> |

Hier ist die Liste der Entscheidungen:
Die Kriterien werden durch das „|“ Symbol getrennt.
{TEXT}
"""

    #The column critera should name the specified criteria, match indicates if the criteria matches or not and the column reference shows the part of the text which corresponds to the criteria. 


In [38]:
%time table_out = aggregate_decisions(outputs, agg_decisions_prompt)

CPU times: user 46.4 ms, sys: 1.64 ms, total: 48.1 ms
Wall time: 2min 1s


In [39]:
file_path=str(story_no)+'.md'
with open(file_path, 'w') as file:
    print(table_out, file = file)
    print(table_out)

| Kriterium | Entscheidung | Referenzen |
| ------------- |:-------------:|:-------------|
| Der Text erwähnt den Evidenzlevel. Was sind die Einschränkungen bei den Studienergebnissen? Gibt es Einschränkungen, die dazu führen, dass die zentralen Ergebnisse fragwürdig sind? | Ja | <ul><li>"Dabei stellten sie fest, dass sich bei geringer Viruslast neun von zehn Hühner nicht infizierten, bei steigender Viruslast die Resistenz jedoch abnahm." (aus dem Text)</li><li>"Eine vollständige Resistenz der Hühner könnte theoretisch erreicht werden, wenn alle drei Gene der Genfamilie ANP32 verändert werden würden. Das zeigten Experimente in Zellkulturen. Das Ausschalten aller drei Gene wäre für ein Huhn jedoch vermutlich tödlich." (aus dem Text)</li><li>"Doch auch hier gibt es Einschränkungen: In dem Leitfaden für die Umweltverträglichkeitsprüfung von gentechnisch veränderten Tieren der EFSA [III] wird auf diverse Risiken hingewiesen: Genetisch veränderte Tiere können eine längere Inkubationszeit ha

In [40]:
display(Markdown(table_out))

| Kriterium | Entscheidung | Referenzen |
| ------------- |:-------------:|:-------------|
| Der Text erwähnt den Evidenzlevel. Was sind die Einschränkungen bei den Studienergebnissen? Gibt es Einschränkungen, die dazu führen, dass die zentralen Ergebnisse fragwürdig sind? | Ja | <ul><li>"Dabei stellten sie fest, dass sich bei geringer Viruslast neun von zehn Hühner nicht infizierten, bei steigender Viruslast die Resistenz jedoch abnahm." (aus dem Text)</li><li>"Eine vollständige Resistenz der Hühner könnte theoretisch erreicht werden, wenn alle drei Gene der Genfamilie ANP32 verändert werden würden. Das zeigten Experimente in Zellkulturen. Das Ausschalten aller drei Gene wäre für ein Huhn jedoch vermutlich tödlich." (aus dem Text)</li><li>"Doch auch hier gibt es Einschränkungen: In dem Leitfaden für die Umweltverträglichkeitsprüfung von gentechnisch veränderten Tieren der EFSA [III] wird auf diverse Risiken hingewiesen: Genetisch veränderte Tiere können eine längere Inkubationszeit haben und sich auch mit Krankheitserregern aktiver verhalten, wodurch mehr Krankheitserreger entstehen und diese länger übertragen werden können. Die infizierten Tiere werden erst spät erkannt und nicht-genetisch veränderte Tiere können verstärkt angesteckt werden. Noch dazu ist dieses Verfahren nicht ausreichend erforscht, um negative gesundheitliche Auswirkungen auf Tier und Mensch auszuschließen. Zudem bleibt die Frage, ob sich bei einer Genom-Editierung das Virus nicht durch weitere Mutationen ausbreiten kann." (aus dem Text)</li><li>"Die initialen Veränderungen (zwei Aminosäureaustausche) verhinderten Infektionen, wenn eine geringe Virusdosis genutzt wurde; bei höheren Dosen allerdings wurde die Resistenz (nicht: Immunität!) durchbrochen, da sich in der Masse der inokulierten Viren bereits Virusvarianten befanden, die Mutationen aufwiesen, um das veränderte ANP32A-Protein zu umgehen." (Aussage von Prof. Dr. Timm Harder)</li><li>"Dies ist ein kritischer Punkt, der darauf hinweisen könnte, dass Viren existieren, die weitere resistenzumgehende Mutationen entwickeln können." (Aussage von Prof. Dr. Timm Harder)</li><li>"Es bleibt abzuwarten, wie sich diese Hühner gegenüber den wesentlich aggressiveren, hochpathogenen aviären Influenzaviren wie H5N1 verhalten; diese wurden hier nicht getestet." (Aussage von Prof. Dr. Timm Harder)</li><li>"Gleichzeitig wird aber auch die enorme Anpassungsfähigkeit der Viren deutlich, die bei hohen Viruslasten schon in diesen ersten Experimenten zu Durchbruchsinfektionen geführt hat." (Aussage von Prof. Dr. Stephan Ludwig)</li><li>"Neben dem in der Arbeit aufgezeigten Problem, das die Viren zu schnellen Anpassungen getrieben werden und so der Ansatz schnell unwirksam werden könnte, sehe ich sowohl rechtliche als auch ethische Hürden, und gerade in der angeblich so aufgeklärten westlichen Welt auch Probleme bei der Akzeptanz in der Bevölkerung." (Aussage von Prof. Dr. Stephan Ludwig)</li></ul> |
| Der Text enthält Informationen über Krebs. | Nein | <ul><li>Der gesamte bereitgestellte Text und die Expertenaussagen beziehen sich auf die Vogelgrippe und genetische Veränderungen bei Hühnern zur Resistenz gegen das Vogelgrippevirus. Es gibt keine Erwähnung von Krebs oder krebsbezogenen Themen im Text oder in den Expertenaussagen.</li></ul> |
| Der Text enthält die Nennung eines spezifischen Zeitschriftentitels einer Studie. | Ja | <ul><li>"Die britischen Forschenden, die ihre Ergebnisse im Fachjournal „Nature Communications” veröffentlichten (siehe Primärquelle), stellen damit eine Methode vor, mit der Nutztierbestände vor der Vogelgrippe geschützt und die Verbreitung des Virus eingedämmt werden könnte." (aus dem Text)</li></ul> |
| Der Text beschreibt eine mathematische Methode zur Optimierung einer Verlustfunktion mit vier Zielen. | Nein | <ul><li>Der Text beschreibt eine Studie, die sich mit der genetischen Veränderung von Hühnern zur Resistenz gegen das Vogelgrippevirus befasst. (aus dem Text)</li><li>Es werden keine mathematischen Methoden oder Verlustfunktionen erwähnt. (aus dem Text)</li><li>Die Expertenstatements beziehen sich auf die biologischen und epidemiologischen Aspekte der Studie und deren mögliche Auswirkungen, enthalten aber keine Informationen über mathematische Optimierungsmethoden. (aus den Expertenstatements von Prof. Dr. Timm Harder und Prof. Dr. Stephan Ludwig)</li></ul> |
| Der Text macht klar, in welchem Forschungsstadium sich die Ergebnisse befinden. Dies lässt Rückschlüsse auf die Verfügbarkeit von beispielsweise neuen Medikamenten oder Behandlungen zu. | Ja | <ul><li>"Bei dieser Arbeit handelt es sich um eine sogenannte Proof-of-Concept-Studie, also eine reine Machbarkeitsstudie." (aus dem Text)</li><li>"Es ist eine Proof-of-Concept-Studie, die zeigt, dass eine gut durchdachte Gene-Editing-Strategie geeignet sein kann, um eine robuste Resistenz gegen Infektion zu erreichen." (Expertenaussage von Prof. Dr. Stephan Ludwig)</li><li>"Ohne die Erkenntnisse der Arbeit schmälern zu wollen, bin ich sehr skeptisch, ob ein solcher Ansatz tatsächlich mittelfristig in der Breite durchführbar ist. [...] Insofern ist die Arbeit zunächst einmal eine elegante akademische Fingerübung und noch weit weg von einer tatsächlichen Anwendung." (Expertenaussage von Prof. Dr. Stephan Ludwig)</li><li>"Dem EU-Recht nach sind Organismen, deren Genom mittels dem CRISPR/Cas-Verfahren verändert wurden, als gentechnisch veränderte Organismen (GMOs) zu betrachten. Ihre Nutzung bedürfte also einer gentechnischen Genehmigung und die Haltung wäre nach aktuellem Recht nur in einer gentechnischen Anlage möglich. Freilandhaltung wäre dann einem Freisetzungsvorhaben gleichzustellen. Ohne entsprechende rechtliche Anpassungen wäre eine Massennutzung sicherlich nicht vorstellbar." (Expertenaussage von Prof. Dr. Timm Harder)</li></ul> |
| Der Text erwähnt das zentrale Forschungsergebnis. | Ja | <ul><li>"Die Forschenden tauschten mittels Genom-Editierung zwei Aminosäuren des Wirtproteins ANP32A aus, welches essenziell für die Vermehrung des Virus in Hühnern ist. Dabei stellten sie fest, dass sich bei geringer Viruslast neun von zehn Hühner nicht infizierten, bei steigender Viruslast die Resistenz jedoch abnahm." (aus dem Text)</li><li>"Eine vollständige Resistenz der Hühner könnte theoretisch erreicht werden, wenn alle drei Gene der Genfamilie ANP32 verändert werden würden. Das zeigten Experimente in Zellkulturen." (aus dem Text)</li><li>"Das Ausschalten aller drei Gene wäre für ein Huhn jedoch vermutlich tödlich." (aus dem Text)</li><li>"Bei dieser Arbeit handelt es sich um eine sogenannte Proof-of-Concept-Studie, also eine reine Machbarkeitsstudie." (aus dem Text)</li><li>Prof. Dr. Timm Harder: "Die initialen Veränderungen (zwei Aminosäureaustausche) verhinderten Infektionen, wenn eine geringe Virusdosis genutzt wurde; bei höheren Dosen allerdings wurde die Resistenz (nicht: Immunität!) durchbrochen, da sich in der Masse der inokulierten Viren bereits Virusvarianten befanden, die Mutationen aufwiesen, um das veränderte ANP32A-Protein zu umgehen." (aus der Expertenaussage von Prof. Dr. Timm Harder)</li><li>Prof. Dr. Stephan Ludwig: "Es handelt sich um eine sehr elegante Arbeit, die die entscheidende Wichtigkeit der ANP32-Genfamilie für die Vermehrung von Vogelgrippeviren in einem komplexen Organismus aufzeigt. Es ist eine Proof-of-Concept-Studie, die zeigt, dass eine gut durchdachte Gene-Editing-Strategie geeignet sein kann, um eine robuste Resistenz gegen Infektion zu erreichen." (aus der Expertenaussage von Prof. Dr. Stephan Ludwig)</li></ul> |